In [56]:
from path import Path
import sys

data_path = Path().joinpath('data')  # You can switch to another base path here

In [53]:
HOST = 'geoftp.ibge.gov.br'
URL = ('organizacao_do_territorio/malhas_territoriais/'
       'malhas_de_setores_censitarios__divisoes_intramunicipais/'
       'censo_2010/setores_censitarios_shp')
LEVELS = {
    'regioes': 0,
    'estados': 1,
    'municipios': 2,
    'distritos': 3,
    'setores': 4
}

#### URLs for the SHP files in IBGE's FTP server

In [54]:
import contextlib
import ftplib
import re

def iter_shps_states(host, url):
    """
    Generates tuples [state, level, url] for the SHP files of Brazilian states
    """
    ftp = ftplib.FTP(host)
    ftp.login()
    
    with contextlib.closing(ftp):
        for state_path in ftp.nlst(url):
            basename = Path(state_path).basename()

            if not re.match('''[a-z]{2}$''', basename):
                continue
            
            state_abbr = basename.upper()
            for fzip in ftp.nlst(state_path):
                zip_basename = Path(fzip).basename()
                level = [level for name, level in LEVELS.items() if name in zip_basename][0]
                yield state_abbr, level, 'ftp://{host}/{fzip}'.format(host=host,fzip=fzip)

SHPS_STATES = list(iter_shps_states(HOST, URL))

#### Initializing the database

In [11]:
import sqlite3

con = sqlite3.connect(data_path.joinpath('dados.db'))
cur = con.cursor()

In [3]:
from jfp_backend.db import schema as db_schema
import imp
imp.reload(db_schema)

db_schema.default_engine = (
    'mysql+mysqlconnector://diogenes:123456'
    '@localhost:3306/generic'
)
SessionMaker, engine = db_schema.get_db_session()
con = engine.connect()

## Inserir as regiões

In [4]:
con.execute(db_schema.Regiao.__table__.insert(),
           [{'nome': nome} for nome in ('Centro-Oeste', 'Sul', 'Nordeste', 'Norte', 'Sudeste')])

In [5]:
session = SessionMaker()
regioes = {regiao.nome: regiao.id for regiao in session.query(db_schema.Regiao)}

In [6]:
regioes

{'Centro-Oeste': 6, 'Nordeste': 8, 'Norte': 9, 'Sudeste': 10, 'Sul': 7}

## Carregar os códigos de todas as geometrias

In [7]:
from shapely.geometry.point import Point
from shapely.wkt import dumps as wkt_dumps, loads as wkt_loads

In [9]:
# Inserir a geometria inexistente

p = Point()
geo = db_schema.Geometria(codigo='codigo_inexistente', poligono=wkt_dumps(p))
session = SessionMaker()
session.add(geo)
session.commit()

In [8]:
session = SessionMaker()
geo_codes = {
    g[0]: g[1]
    for g in session
             .query(db_schema.Geometria)
             .with_entities(db_schema.Geometria.codigo, db_schema.Geometria.id)
}

In [9]:
geometria_invalida = [
    {'id': geo[0], 'codigo': geo[1]}
    for geo in session
    .query(db_schema.Geometria)
    .filter(db_schema.Geometria.codigo == 'codigo_inexistente')
    .with_entities(db_schema.Geometria.id, db_schema.Geometria.codigo)
]

In [10]:
geometria_invalida

[{'codigo': 'codigo_inexistente', 'id': 342563}]

In [11]:
id_geometria_invalida = geometria_invalida[0]['id']
id_geometria_invalida

342563

## Inserir os estados

In [14]:
import pandas
estados_df = pandas.read_csv(data_path.joinpath('detalhes_estados.csv'))
estados_df[:4]

,nome,sigla,regiao,codigo,id_regiao
0,Acre,AC,Norte,0,0
1,Alagoas,AL,Nordeste,0,0
2,Amapá,AP,Norte,0,0
3,Amazonas,AM,Norte,0,0


In [15]:
for estado, urls in zips_estados.items():
    
    dir_fname = ibge_path.joinpath('{}_municipios'.format(estado.lower()))
    shp_fname = [fname.basename() for fname in dir_fname.glob('*.shp') if fname.lower().endswith('shp')][0]
    codigo = shp_fname[:2]
    
    estados_df = estados_df.set_value(estados_df['sigla'] == estado, 'codigo', codigo)

In [16]:
estados_df['id_regiao'] = estados_df['regiao'].map(lambda regiao: regioes[regiao])
estados_df['id_geometria'] = id_geometria_invalida

In [17]:
estados_df

,nome,sigla,regiao,codigo,id_regiao,id_geometria
0,Acre,AC,Norte,12,9,342563
1,Alagoas,AL,Nordeste,27,8,342563
2,Amapá,AP,Norte,16,9,342563
3,Amazonas,AM,Norte,13,9,342563
4,Bahia,BA,Nordeste,29,8,342563
5,Ceará,CE,Nordeste,23,8,342563
6,Distrito Federal,DF,Centro-Oeste,53,6,342563
7,Espírito Santo,ES,Sudeste,32,10,342563
8,Goiás,GO,Centro-Oeste,52,6,342563
9,Maranhão,MA,Nordeste,21,8,342563


In [18]:
estados_sql = estados_df.drop(['regiao'], axis=1).to_dict(orient='records')

con.execute(db_schema.Estado.__table__.insert(), estados_sql)

In [19]:
session = SessionMaker()
estados_codigos = {estado.codigo: estado.id
                   for estado in session.query(db_schema.Estado)}

In [20]:
estados_codigos

{'11': 49,
 '12': 28,
 '13': 31,
 '14': 50,
 '15': 41,
 '16': 30,
 '17': 54,
 '21': 37,
 '22': 45,
 '23': 33,
 '24': 47,
 '25': 42,
 '26': 44,
 '27': 29,
 '28': 53,
 '29': 32,
 '31': 40,
 '32': 35,
 '33': 46,
 '35': 52,
 '41': 43,
 '42': 51,
 '43': 48,
 '50': 39,
 '51': 38,
 '52': 36,
 '53': 34}

## Extrair os códigos e nomes de todas as subdivisões

In [21]:
campos = {
    'municipios': {'ibge_codigo': 'CD_GEOCODM', 'ibge_nome': 'NM_MUNICIP',
                   'db_nome': 'nome'},
    'distritos': {'ibge_codigo': 'CD_GEOCODD', 'ibge_nome': 'NM_DISTRIT',
                  'db_nome': 'nome'},
    'subdistritos': {'ibge_codigo': 'CD_GEOCODS', 'ibge_nome': 'NM_SUBDIST',
                     'db_nome': 'nome'},
    'setores': {'ibge_codigo': 'CD_GEOCODI', 'ibge_nome': 'NM_BAIRRO',
                'db_nome': 'bairro'}
}

In [ ]:
import geopandas

pbar = tqdm_notebook(total=27, desc='Obtendo')

for estado, info in zips_estados.items():
    estado_info = {}
    
    if ibge_path.joinpath(estado+'.json').isfile():
        pbar.update(1)
        continue
    
    for divisao, path in info.items():
        dir_fname = ibge_path.joinpath('{}_{}'.format(estado.lower(), divisao))
        shp_fname = [fname for fname in dir_fname.glob('*.shp') if fname.lower().endswith('shp')][0]
        
        df = geopandas.read_file(shp_fname)
        
        ibge_codigos = list(map(str, df[campos[divisao]['ibge_codigo']]))
        ibge_nomes = list(map(str, df[campos[divisao]['ibge_nome']]))
        
        estado_info[divisao] = [
            dict(zip(['codigo', campos[divisao]['db_nome']], item))
            for item in zip(ibge_codigos, ibge_nomes)
        ]
    
    with open(ibge_path.joinpath(estado+'.json'), 'w') as fp:
        json.dump(estado_info, fp, indent=2)
    
    pbar.update(1)

pbar.close()

## Inserir os municípios

In [23]:
from tqdm import tqdm_notebook
import json

municipios_sem_geometria = []

for estado_path in tqdm_notebook(ibge_path.glob('*.json')):
    estado = estado_path.basename().splitext()[0]
    with open(estado_path) as fp:
        estado_info = json.load(fp)
    
    municipios_estado = estado_info['municipios']
    municipios_estado_sql = []
    
    for municipio in municipios_estado:
        municipio['id_estado'] = estados_codigos[municipio['codigo'][:2]]
        codigo_municipio = municipio['codigo']
        try:
            id_geometria = geo_codes[codigo_municipio]
        except KeyError:
            id_geometria = id_geometria_invalida
        municipio['id_geometria'] = id_geometria
        if id_geometria == id_geometria_invalida:
            municipios_sem_geometria.append(municipio)
        municipios_estado_sql.append(municipio)
    
    con.execute(db_schema.Municipio.__table__.insert(), municipios_estado_sql)

with open(data_path.joinpath('sem-geometria-municipios.json'), 'w') as fp:
    json.dump(municipios_sem_geometria, fp, indent=2)

A Jupyter Widget

In [36]:
session = SessionMaker()
municipios_codigos = {mun.codigo: mun.id for mun in session.query(db_schema.Municipio)}
len(municipios_codigos)

5567

## Inserir os distritos

In [25]:
len(list(municipios_codigos)[0])

7

In [26]:
distritos_sem_geometria = []

for estado_path in tqdm_notebook(ibge_path.glob('*.json')):
    estado = estado_path.basename().splitext()[0]
    with open(estado_path) as fp:
        estado_info = json.load(fp)
    
    distritos_estado = estado_info['distritos']
    distrito_estado_sql = []
    
    for distrito in distritos_estado:
        distrito['id_municipio'] = municipios_codigos[distrito['codigo'][:7]]
        codigo_distrito = distrito['codigo']
        try:
            id_geometria = geo_codes[distrito['codigo']]
        except KeyError:
            id_geometria = id_geometria_invalida
        distrito['id_geometria'] = id_geometria
        if id_geometria == id_geometria_invalida:
            distritos_sem_geometria.append(distrito)
        distrito_estado_sql.append(distrito)
    
    con.execute(db_schema.Distrito.__table__.insert(), distrito_estado_sql)

with open(data_path.joinpath('sem-geometria-distritos.json'), 'w') as fp:
    json.dump(distritos_sem_geometria, fp, indent=2)

A Jupyter Widget

In [35]:
session = SessionMaker()
distritos_codigos = {dis.codigo: dis.id for dis in session.query(db_schema.Distrito)}
len(distritos_codigos)

10283

## Inserir os subdistritos

In [28]:
len(list(distritos_codigos)[0])

9

In [29]:
subdistritos_sem_geometria = []

for estado_path in tqdm_notebook(ibge_path.glob('*.json')):
    estado = estado_path.basename().splitext()[0]
    with open(estado_path) as fp:
        estado_info = json.load(fp)
    
    subdistritos_estado = estado_info['subdistritos']
    subdistrito_estado_sql = []
    
    for subdistrito in subdistritos_estado:
        subdistrito['id_distrito'] = distritos_codigos[subdistrito['codigo'][:9]]
        codigo_subdistrito = subdistrito['codigo']
        
        try:
            id_geometria = geo_codes[codigo_subdistrito]
        except KeyError:
            id_geometria = id_geometria_invalida
        subdistrito['id_geometria'] = id_geometria
        if id_geometria == id_geometria_invalida:
            subdistritos_sem_geometria.append(subdistrito)
        subdistrito_estado_sql.append(subdistrito)
    
    con.execute(db_schema.Subdistrito.__table__.insert(), subdistrito_estado_sql)

with open(data_path.joinpath('sem-geometria-subdistritos.json'), 'w') as fp:
    json.dump(subdistritos_sem_geometria, fp, indent=2)

A Jupyter Widget

In [4]:
session = SessionMaker()
subdistritos_codigos = {subdis.codigo: subdis.id for subdis in session.query(db_schema.Subdistrito)}
len(subdistritos_codigos)

10908

## Inserir os setores

In [5]:
len(list(subdistritos_codigos)[0])

11

In [13]:
from tqdm import tqdm_notebook
setores_sem_geometria = []

for estado_path in tqdm_notebook(ibge_path.glob('*.json')):
    estado = estado_path.basename().splitext()[0]
    with open(estado_path) as fp:
        estado_info = json.load(fp)
    
    setores_estado = estado_info['setores']
    setor_estado_sql = []
    
    for setor in setores_estado:
        setor['id_subdistrito'] = subdistritos_codigos[setor['codigo'][:11]]
        codigo_setor = setor['codigo']
        try:
            id_geometria = geo_codes[codigo_setor]
        except KeyError:
            id_geometria = id_geometria_invalida
        setor['id_geometria'] = id_geometria
        if id_geometria == id_geometria_invalida:
            setores_sem_geometria.append(setor)
        else:
            # Não faz sentido ter setores sem geometria
            setor_estado_sql.append(setor)
    
    con.execute(db_schema.Setor.__table__.insert(), setor_estado_sql)
    
with open(data_path.joinpath('sem-geometria-setores.json'), 'w') as fp:
    json.dump(setores_sem_geometria, fp, indent=2)

A Jupyter Widget

In [14]:
session = SessionMaker()
setores_codigos = {setor.codigo: setor.id for setor in session.query(db_schema.Setor)}
len(setores_codigos)

316155

In [15]:
len(setores_sem_geometria)

419